This version of the cascaded ST model does not contain any additional components (punctuation/disfluency)

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install bert-score transformers datasets[audio] sentencepiece sacrebleu sacremoses torchmetrics

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForMaskedLM, Wav2Vec2Model, Wav2Vec2Processor, Wav2Vec2ForCTC, AutoModelForSeq2SeqLM
from torch import argmax
from torch import stack, cat, no_grad
import torch.optim as optim 

import pandas as pd
from datasets import Dataset
import numpy as np
from math import ceil
import torchaudio
import torch.nn.functional as F
from sacrebleu.metrics import BLEU
import re
import yaml

In [ ]:
%cd /
%cp -r '/content/drive/MyDrive/Colab Notebooks/huggingface' '/root/.cache/'

/


In [ ]:
audio_folder_path = '/content/drive/MyDrive/en-de/data/train/wav/'

# Dataset

## Utility functions

In [ ]:
def remove_special_characters(s):
    chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\—]'
    s = re.sub(chars_to_ignore_regex, '', s).lower()
    s = re.sub(r'[0-9]+', '', s)
    s = re.sub(' +', ' ', s)
    return s

def prepare_audio_batch(batch, device):
  batch_audio = []
  for i in range(len(batch['file'])):
    data = {'file': batch['file'][i], 
              'start': batch['start'][i].item(),
              'end': batch['end'][i].item(),
              'raw_text': batch['raw_text'][i],
              'text': batch['text'][i],
            }
    audio_segment = getAudioSegment(data, audio_folder_path)
    batch_audio.append(audio_segment)
  return batch_audio

def getAudioSegment(data, parent_path):
    path = parent_path + data['file']
    audio_tensor, rate = torchaudio.load(path)
    audio = audio_tensor.numpy().transpose().flatten()
    # audio = torch.FloatTensor(audio[data['start']:data['end']])
    audio = audio[data['start']:data['end']]
    del audio_tensor, path
    return audio

def read_yaml(path):
  yaml_file = None
  with open(path, "r") as stream:
      try:
          yaml_file = yaml.safe_load(stream)
      except yaml.YAMLError as exc:
          print(exc)
  return yaml_file

## Loading data

Input data

In [ ]:
path = "/content/drive/MyDrive/en-de/data/train/txt/train.yaml"
yaml_file = read_yaml(path)
en_file = open("/content/drive/MyDrive/en-de/data/train/txt/train.en", "r")
de_file = open("/content/drive/MyDrive/en-de/data/train/txt/train.de", "r")

current_wav = ""
audio, rate = None, 16000
train_data_list = []

cnt = -1
for line_props in yaml_file:
  english_text = en_file.readline()[:-2]
  deutsch_text = de_file.readline()[:-2]
  cnt += 1

  entry_dict = {}
  if line_props['wav'] != current_wav:
    current_wav = line_props['wav']
  
  punctuated_text = english_text
  reference = remove_special_characters(punctuated_text)
  if(len(reference.split(' ')) < 6):
      continue

  offset = line_props['offset']
  duration = line_props['duration']
  start = int(offset*rate)
  end = int( ceil( (offset*rate)+(duration*rate) ))

  entry_dict['file'] = current_wav
  entry_dict['start'] = start
  entry_dict['end'] = end
  entry_dict['raw_text'] = punctuated_text
  entry_dict['text'] = reference
  entry_dict['german'] = deutsch_text
  train_data_list.append(entry_dict)

In [ ]:
# df_all = pd.DataFrame(train_data_list)
# df_all.to_csv('/content/drive/MyDrive/en-de/all_chosen_data_eval_en-de.csv')
# train_data_list = None

df_all = pd.read_csv('/content/drive/MyDrive/en-de/all_chosen_data_eval_en-de.csv')
df_all.drop(axis=1, columns = [df_all.columns[0]], inplace=True)
df_all.head() 

,file,start,end,raw_text,text,german
0,ted_1.wav,98720,559520,And it's truly a great honor to have the oppor...,and it's truly a great honor to have the oppor...,"Vielen Dank, Chris. Es ist mir wirklich eine E..."
1,ted_1.wav,560160,779200,"And I say that sincerely, partly because (Mock...",and i say that sincerely partly because (mock ...,"Das meine ich ernst, teilweise deshalb — weil ..."
2,ted_1.wav,779200,1146400,Now I have to take off my shoes or boots to ge...,now i have to take off my shoes or boots to ge...,"Jetzt muss ich meine Schuhe ausziehen, um über..."
3,ted_1.wav,1161600,1227520,I'll tell you one quick story to illustrate wh...,i'll tell you one quick story to illustrate wh...,"Ich erzähle Ihnen mal eine Geschichte, dann ve..."
4,ted_1.wav,1235520,1363840,It's a true story — every bit of this is true....,it's a true story every bit of this is true so...,Eine wahre Geschichte — kein Wort daran ist er...


In [ ]:
# taking 3k sentences for evaluation
df_eval = df_all[20000:25000]

eval_dataset = Dataset.from_pandas(df_eval)
eval_dataset = eval_dataset.shuffle(seed = 42)
print(eval_dataset)

Dataset({
    features: ['file', 'start', 'end', 'raw_text', 'text', 'german'],
    num_rows: 5000
})


In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(eval_dataset))

print(eval_dataset[rand_int]["raw_text"])
print(eval_dataset[rand_int]["german"])

rndm_audio = getAudioSegment(eval_dataset[rand_int], audio_folder_path)
ipd.Audio(data=np.asarray(rndm_audio), autoplay=True, rate=16000)

Or you can view it as an evolutionary psychology kind of thing: did our genes invent this as a kind of trick to get us to behave in certain ways
Oder man kann es als eine Sache der evolutionären Psychologie sehen. Warum sollten unsere — haben unsere Gene das als eine Art Trick erfunden damit wir uns auf bestimmte Weisen verhalten


## Calculating length of training subset

In [ ]:
df_train = df_all[:15000]
df_train['length'] = (df_train.end - df_train.start)/16000

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


99301.609125

In [ ]:
df_train.length.min()

0.05

In [ ]:
df_train.length.sum()/3600

27.5837803125

In [ ]:
df_eval = df_all[20000:25000]
df_eval['length'] = (df_eval.end - df_eval.start)/16000
df_eval.length.sum()/3600

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


9.69858293402778

# Model

In [ ]:
class ASR(nn.Module):
    def __init__(self, model_path = "facebook/wav2vec2-base-960h") -> None:
        super(ASR, self).__init__()

        self.processor = Wav2Vec2Processor.from_pretrained(model_path)
        self.model = Wav2Vec2ForCTC.from_pretrained(model_path)

        if torch.cuda.is_available():
          self.model = self.model.cuda()
    
    def forward(self, audio):
        input_values = self.processor(audio, sampling_rate=16000, return_tensors='pt').input_values
        if torch.cuda.is_available():
          input_values = input_values.cuda()
        logits = self.model(input_values).logits
        predicted_ids = argmax(logits, dim=-1)
        transcription = self.processor.decode(predicted_ids[0])

        del input_values, logits, predicted_ids
        return transcription
    

class Translator(nn.Module):
    def __init__(self, model_path = "facebook/wmt19-en-de"):
        super(Translator, self).__init__()

        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

        if torch.cuda.is_available():
          self.model = self.model.cuda()

    def forward(self, text):
        input_ids = self.tokenizer(text, return_tensors = 'pt').input_ids
        if torch.cuda.is_available():
          input_ids = input_ids.cuda()
        outputs = self.model.generate(input_ids)
        translation = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        del input_ids, outputs

        return translation

# class MUSE

# class DisfluencyDetector

class CascadedST(nn.Module):
    def __init__(self, with_punct = False, with_disf = False):
        super(CascadedST, self).__init__()

        if torch.cuda.is_available():
          self.asr = ASR().cuda()
        
        self.with_punct = with_punct
        # if with_punct:
        #     self.muse_model = MUSE() 

        self.with_disf = with_disf
        # if with_disf:
        #     self.disfluency_detector = DisfluencyDetector()

        if torch.cuda.is_available():
          self.translator = Translator().cuda()
        if torch.cuda.is_available():
          self.asr = self.asr.cuda()
          self.translator = self.translator.cuda()
        

    def forward(self, audio):
        transcript = self.asr(audio)

        # if self.with_punct:
            # transcript = self.muse_model(transcript, audio)
        # if self.with_disf:
            # transcript = self.disfluency_detector(transcript)

        translation = self.translator(transcript)
        
        return translation

#Trying things first

In [ ]:
dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=1)
batch = next(iter(dataloader))

In [ ]:
audio = torch.Tensor(prepare_audio_batch(batch, device)[0]).cuda()

In [ ]:
asr = ASR().to(device)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
transcript = asr(audio)
transcript

'WOMEN SIT AT TABLES AND MEN GO FROM TABLE TO TABLE NEARS A CLOCK AND A BELL IN AT FIVE MINUTES MINGO THE CONVERSATION ENDS AND THE WOMAN CAN DECIDE WHETHER TO GIVE HER CARD OR HER EMALE ADDRESSED THE MAN FOR FALAP'

In [ ]:
translator = Translator().to(device)

In [ ]:
translation = translator(transcript)

In [ ]:
print(translation)
print(batch['german'])

FRAUEN, die an Tischen sitzen und von Tisch zu Tisch gehen, um in der Nähe einer Uhr und einer Glöckchen in fünf Minuten die Konversation zu beenden und die FRAU kann entscheiden, WENN sie ihre Karte oder ihre E-Mail an den Mann für FalAP geben soll
['Frauen sitzen an Tischen und Männer gehen von Tisch zu Tisch, und es gibt eine Uhr und eine Glocke und nach fünf Minuten endet das Gespräch und die Frau kann entscheiden, ob sie ihm ihre Karte oder E-Mail-Adresse gibt, zum weiteren Kontakt']


In [ ]:
bleu = BLEU(lowercase=True)
score = bleu.corpus_score([translation], [batch['german']])
score

BLEU = 20.77 53.1/27.1/14.9/8.7 (BP = 1.000 ratio = 1.089 hyp_len = 49 ref_len = 45)

In [ ]:
from torchmetrics import SacreBLEUScore
metric = SacreBLEUScore(smooth=True, lowercase = True)
metric([translation], [batch['german']])

tensor(0.2077)

# Evaluating model variants

In [ ]:
def eval_model(model, dataset, device):
  dataloader = torch.utils.data.DataLoader(dataset, batch_size=1)
  translations = []
  references = []
  for idx, batch in enumerate(dataloader):
    if idx%500==0:
      print(f'current batch = {idx}')
    try:
      audio = prepare_audio_batch(batch, device)[0]
      translated_text = model(audio)
      translations.append(translated_text)
      references.append(batch['german'][0])
    except Exception as ex:
      print(idx, ex)
  bleu = BLEU(lowercase=True)
  score = bleu.corpus_score(translations, [references])

  return translations, score

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Vanilla variant

In [ ]:
vanilla_model = CascadedST().to(device)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
translations, vanilla_score = eval_model(vanilla_model, eval_dataset, device)
print(vanilla_score)

current batch = 0
current batch = 500
current batch = 1000
current batch = 1500
current batch = 2000
current batch = 2500
current batch = 3000
current batch = 3500
current batch = 4000
current batch = 4500
BLEU = 6.74 34.0/10.7/4.3/1.9 (BP = 0.919 ratio = 0.922 hyp_len = 96684 ref_len = 104859)


BLEU = 6.74 34.0/10.7/4.3/1.9 (BP = 0.919 ratio = 0.922 hyp_len = 96684 ref_len = 104859)

In [ ]:
from torchmetrics import SacreBLEUScore
metric = SacreBLEUScore(smooth=True, lowercase = True)
metric(translations,[eval_dataset['german']])

tensor(0.4329)

With smoothing and lowercasing: 0.4329